In [1]:
def hamming_encode(data):
    # Предполагаем, что data - это строка из 4 бит, например, "1001"
    data = list(map(int, data))
    p1 = data[0] ^ data[1] ^ data[3]
    p2 = data[0] ^ data[2] ^ data[3]
    p3 = data[1] ^ data[2] ^ data[3]
    return [p1, p2, data[0], p3, data[1], data[2], data[3]]

def hamming_decode(data):
    # Предполагаем, что data - это строка из 7 бит, например, "0111001"
    data = list(map(int, data))
    p1 = data[0]
    p2 = data[1]
    d1 = data[2]
    p3 = data[3]
    d2 = data[4]
    d3 = data[5]
    d4 = data[6]
    s1 = p1 ^ d1 ^ d2 ^ d4
    s2 = p2 ^ d1 ^ d3 ^ d4
    s3 = p3 ^ d2 ^ d3 ^ d4
    error_position = s1 * 1 + s2 * 2 + s3 * 4
    if error_position != 0:
        data[error_position - 1] ^= 1
    return data[2], data[4], data[5], data[6]

data = "1001"
encoded_data = hamming_encode(data)
print("Encoded data:", ''.join(map(str, encoded_data)))

# Вводим ошибку для тестирования декодирования
encoded_data[3] ^= 1  # Инвертируем один бит для имитации ошибки

decoded_data = hamming_decode(encoded_data)
print("Decoded data:", ''.join(map(str, decoded_data)))


Encoded data: 0011001
Decoded data: 1001


In [8]:
# Установка библиотеки reedsolo
!pip install reedsolo

# Импорт библиотеки и исключений
from reedsolo import RSCodec, ReedSolomonError

# Инициализация кода Рида-Соломона с 10 контрольными символами
rsc = RSCodec(10)  # 10 контрольных байтов

# Исходные данные
data = "Hello, World!"

# Кодирование данных
rs_encoded = rsc.encode(data.encode('utf-8'))
print("Encoded data (RS):", rs_encoded)

# Введение ошибок для тестирования декодирования
# Например, изменяем 3-й и 7-й байты
rs_encoded = bytearray(rs_encoded)
rs_encoded[3] ^= 0xFF
rs_encoded[7] ^= 0xFF

try:
    rs_decoded = rsc.decode(rs_encoded)
    # Получаем только декодированные данные из кортежа
    decoded_data = rs_decoded[0].decode('utf-8')
    print("Decoded data:", decoded_data)
except ReedSolomonError as e:
    print("Error during decoding:", e)


Encoded data (RS): bytearray(b'Hello, World!\xac\x16\xcc-\xd7\x99\xe3<\xbfX')
Decoded data: Hello, World!


In [22]:
# Установка библиотеки reedsolo


# Импорт необходимых модулей
from reedsolo import RSCodec, ReedSolomonError

# Инициализация кода Рида-Соломона
rsc = RSCodec(10)  # 10 контрольных байтов

def hamming_encode(data):
    # Реализация кода Хэмминга
    data = list(map(int, data))
    p1 = data[0] ^ data[1] ^ data[3]
    p2 = data[0] ^ data[2] ^ data[3]
    p3 = data[1] ^ data[2] ^ data[3]
    return [p1, p2, data[0], p3, data[1], data[2], data[3]]

def hamming_decode(data):
    # Реализация декодирования кода Хэмминга
    data = list(map(int, data))
    p1 = data[0]
    p2 = data[1]
    d1 = data[2]
    p3 = data[3]
    d2 = data[4]
    d3 = data[5]
    d4 = data[6]
    s1 = p1 ^ d1 ^ d2 ^ d4
    s2 = p2 ^ d1 ^ d3 ^ d4
    s3 = p3 ^ d2 ^ d3 ^ d4
    error_position = s1 * 1 + s2 * 2 + s3 * 4
    if error_position != 0:
        data[error_position - 1] ^= 1
    return [data[2], data[4], data[5], data[6]]

def cascade_encode(data):
    # Каскадное кодирование
    rs_encoded = rsc.encode(data.encode('utf-8'))  # Кодирование Рида-Соломона
    hamming_encoded = []  # Кодирование Хэмминга
    for byte in rs_encoded:
        hamming_encoded.extend(hamming_encode(f"{byte:08b}"))
    return hamming_encoded

# Пример данных
data = "Hello, World!"

# Кодирование данных
encoded_data = cascade_encode(data)
print("Encoded data (RS + Hamming):", ''.join(map(str, encoded_data)))

Encoded data (RS + Hamming): 10011001100110110011011001101100110010101001010100100101110011000011111100110110011001010101011010110100101111000101010101010100110010010110100001101100110100101
